In [10]:
import networkx as nx
import cenpy
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from matplotlib.pyplot import figure

In [ ]:
# osmnx
pioneer_valley = ['Hampshire County, Massachusetts, USA', 'Hampden County, Massachusetts, USA', 'Franklin County, Massachusetts, USA']
graph = ox.graph_from_place(pioneer_valley, network_type='drive', simplify=False)
area = ox.geocode_to_gdf(pioneer_valley)

# Compute betweenness centrality using networkx
bc = nx.betweenness_centrality(graph, weight='length')
nodes_betweenness_centrality = pd.DataFrame.from_dict(bc, orient='index', columns=['betweenness_centrality'])
nodes_betweenness_centrality.index.name = 'osmid'

pv_nodes, pv_streets  = ox.graph_to_gdfs(graph)
pv_nodes.reset_index(inplace=True)
merged_pv_nodes = pd.merge(pv_nodes, nodes_betweenness_centrality, on='osmid')

In [ ]:
# cenpy
acs = cenpy.products.ACS(2017)

variables = ['B01001_001E', 'B19025A_001E', 'B01002_001E']
variables_info = acs.variables.loc[variables]

for variable in variables:
    print(f"{variable}:")
    print(f"  Label: {variables_info.loc[variable, 'label']}")
    print(f"  Concept: {variables_info.loc[variable, 'concept']}")

spfld_msa_demog = cenpy.products.ACS(2017).from_msa('Springfield, MA', variables=['B01001_001E','B19025A_001E','B01002_001E'])
franklin_demog = cenpy.products.ACS(2017).from_msa('Franklin, MA', variables=['B01001_001E','B19025A_001E','B01002_001E'])

# Compute population density
spfld_msa_demog['population_density_psqkm'] = 1000000 * spfld_msa_demog['B01001_001E'] / spfld_msa_demog.area
franklin_demog['population_density_psqkm'] = 1000000 * franklin_demog['B01001_001E'] / franklin_demog.area

pv_demog = pd.concat([spfld_msa_demog, franklin_demog])

# Calculate inverse income
pv_demog['inverse_income'] = 1 / pv_demog['B19025A_001E']

def normalize(column):
    return (column - column.min()) / (column.max() - column.min())

# Normalize population density and inverse income
pv_demog['population_density_normalized'] = normalize(pv_demog['population_density_psqkm'])
pv_demog['inverse_income_normalized'] = normalize(pv_demog['inverse_income'])

In [ ]:
# megre cenpy and osmnx
merged_pv_nodes = merged_pv_nodes.to_crs(pv_demog.crs)
pv_nodes_demog_joint = merged_pv_nodes.sjoin(pv_demog)

pv_nodes_demog_joint['weighted_betweenness_centrality_popden']=pv_nodes_demog_joint['betweenness_centrality']*pv_nodes_demog_joint['population_density_normalized']
pv_nodes_demog_joint['weighted_betweenness_centrality_income']=pv_nodes_demog_joint['betweenness_centrality']*pv_nodes_demog_joint['inverse_income_normalized']

In [ ]:
# plot equity weighted betweenness_centrality (popden)
fig, ax = plt.subplots(figsize=(15,15))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
pv_nodes_demog_joint.plot('weighted_betweenness_centrality_popden', cmap='plasma', alpha=0.3, ax=ax, linewidth=.5, edgecolor='k',legend=True, cax=cax)
ax.axis('off')
cax.tick_params(labelsize='20')
plt.tight_layout()
plt.savefig('../figures/pv-popdens-equity-weighted-node-betweenness_centrality.png',dpi=120)

In [ ]:
# plot equity weighted betweenness_centrality (inverse income)
fig, ax = plt.subplots(figsize=(15,15))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
pv_nodes_demog_joint.plot('weighted_betweenness_centrality_income', cmap='plasma', alpha=0.3, ax=ax, linewidth=.5, edgecolor='k',legend=True, cax=cax)
ax.axis('off')
cax.tick_params(labelsize='20')
plt.tight_layout()
plt.savefig('../figures/pv-income-equity-weighted-node-betweenness_centrality.png',dpi=120)

In [ ]:
pv_nodes_demog_joint['weighted_betweenness_centrality_popden_income']=(0.5*pv_nodes_demog_joint['population_density_normalized']+0.5*pv_nodes_demog_joint['inverse_income_normalized'])*pv_nodes_demog_joint['betweenness_centrality']